In [13]:
import pandas as pd

# Aqui esta la base de datos 
dataset1 = 'Dataset1erEntrega/Anime.csv'

# Aqui se transforma en Parquet
set1 = pd.read_csv(dataset1)
set1.to_parquet('gralAnimes.parquet')

# Aqui se lo imprime
print(set1.shape)
set1.head()


(16621, 17)


,Anime-PlanetID,Name,Alternative Name,Rating Score,Number Votes,Tags,Content Warning,Type,Episodes,Finished,Duration,StartYear,EndYear,Season,Studios,Synopsis,Url
0,10,The Prince of Tennis,Tennis no Ouji-sama,4.037,10889,"Comedy, Drama, Shounen, Sports, Tennis, Based ...",Unknown,TV,178,True,Unknown,2001,2005,Fall 2001,"Production I.G, Trans Arts","Meet Ryoma Echizen, the cocky prince of tennis...",https://www.anime-planet.com/anime/the-prince-...
1,100,Neon Genesis Evangelion,Shinseiki Evangelion,4.248,54463,"Drama, Mecha, Sci Fi, Conspiracy, Kaijuu, Lone...","Emotional Abuse, Explicit Violence, Mature The...",TV,26,True,Unknown,1995,1996,Fall 1995,"GAINAX, Tatsunoko Production","In the future, a devastating event known as Se...",https://www.anime-planet.com/anime/neon-genesi...
2,1000,Full Metal Panic! The Second Raid,Unknown,4.35,23948,"Action, Comedy, Mecha, Sci Fi, Shounen, Milita...","Explicit Violence, Nudity",TV,13,True,Unknown,2005,2005,Summer 2005,Kyoto Animation,Half a year has passed since Sousuke Sagara to...,https://www.anime-planet.com/anime/full-metal-...
3,10000,22/7: Shampoo no Nioi ga Shita,Unknown,2.8,131,"Idols, School Life, CG Animation",Unknown,Music,1,True,5,2018,2018,Unknown,Unknown,The idol group 22/7 perform the song Shampoo n...,https://www.anime-planet.com/anime/22-7-shampo...
4,10001,Oshiri Tantei: Puputto Fumutto Kaiketsu Dance,Unknown,1.271,21,Family Friendly,Unknown,Web,1,True,2,2017,2017,Unknown,Toei Animation,No synopsis yet - check back soon!,https://www.anime-planet.com/anime/oshiri-tant...


## Recomendaciones de Anime
Esta conjunto de datos contiene informacion sobre 16.621 animes, en el cual podemos encontrar recomendaciones y preferencias de 74.000 usuarios de diferentes animes. De esta manera, podemos extraer cuales son los generos mas vistos, el ranking por puntaje promedio. Asi tambien, lista de animes recomendados y numero de usuarios de acuerdo con la recomendacion.

## Variables

Anime-PlanetID: Anime Planet ID del anime. (por ejemplo, 1).

Nombre: nombre completo del anime. (por ejemplo, FLCL)

Nombre alternativo: otra forma de llamar al anime. (por ejemplo, Furi Kuri)

Puntaje de calificación: puntaje promedio del anime otorgado por todos los usuarios en la base de datos de Anime Planet. (por ejemplo, 8,78)

Número de votos: número de usuarios que le dan una puntuación al anime. (por ejemplo, 1241)

Etiquetas: lista separada por comas de etiquetas para este anime. (por ejemplo, comedia, mecha, ciencia ficción, espacio exterior, trabajo original)

Advertencia de contenido: lista separada por comas de etiquetas de advertencia de contenido. (p. ej., violencia explícita, temas para adultos, desnudez)

Tipo: TV, película, OVA, etc. (ej. TV).

Episodios: número de capítulos. (por ejemplo, 26)

Terminado: Cierto si el anime terminó cuando hice el raspado de datos. Falso es que el anime continúa en ese momento.

Duración: duración del anime en minutos (por ejemplo, 60)

StartYear: año en que el anime inicia la transmisión. (por ejemplo, 2016)

EndYear: año en que el anime finaliza la transmisión. (por ejemplo, 2017)

Temporada: temporada y año de lanzamiento (p. ej., otoño de 2000)

Estudios: lista de estudios separados por comas (p. ej., Sunrise)

Sinopsis: sinopsis del anime.



#### Las variables mas interesantes para el tipo de analisis que podriamos realizar serian "Puntaje de calificacion", "Numero de votos" y "Tipo".

Fuente: Kaggle.com

https://www.kaggle.com/datasets/hernan4444/animeplanet-recommendation-database-2020


In [1]:
import pandas as pd

# Aqui esta la base de datos 
dataset2 = 'Dataset1erEntrega/AppStore.csv'

# Aqui se transforma en Parquet
set2 = pd.read_csv(dataset2)
set2.to_parquet('gralAppStore.parquet')

# Aqui se lo imprime
print(set2.shape)
set2.head()

(7197, 17)


,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1


## Tienda de Aplicaciones Moviles
Este conjunto de datos contiene mas de 7000 detalles de la aplicacion movil Apple iOS, y se ha vuelto muy importante para obtener las aplicaciones mas populares de la tienda. Tambien podemos analizar los precios y ver que género tiene el más elevado; y las calificaciones de los usuarios segun la version. 
  
## Variables

"id" : ID de la aplicación

"track_name": Nombre de la aplicación

"size_bytes": tamaño (en bytes)

"moneda": tipo de moneda

"precio": Importe del precio

"rating_count_tot": recuentos de calificaciones de usuarios (para todas las versiones)

"rating_count_ver": Recuentos de calificaciones de usuarios (para la versión actual)

"user_rating": valor promedio de calificación de usuario (para todas las versiones)

"user_rating_ver": valor promedio de calificación de usuario (para la versión actual)

"ver" : código de la última versión

"cont_rating": Clasificación de contenido

"prime_genre": género principal

"sup_devices.num": Número de dispositivos compatibles

"ipadSc_urls.num": Número de capturas de pantalla mostradas para mostrar

"lang.num": Número de idiomas admitidos

"vpp_lic": licencias basadas en dispositivos Vpp habilitadas

#### Las variables mas interesantes para el analisis podrian ser "Precio", "Moneda", tambien la clasificacion del contenido ("cont_rating"), "prime_genre" para determinar el genero mas descargado.

Fuente: Kaggle.com

https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps

In [2]:
import pandas as pd

# Aqui esta la base de datos 
dataset3 = 'Dataset1erEntrega/HotelReservations.csv'

# Aqui se transforma en Parquet
set3 = pd.read_csv(dataset3)
set3.to_parquet('gralHotel.parquet')

# Aqui se lo imprime
print(set3.shape)
set3.head()

(36275, 19)


,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


## Reservas de Hotel
Este conjunto de datos de reservas de hotel contiene informacion sobre los precios promedios de las habitaciones y de las reservas; el numero de reservas anteriores canceladas por el cliente; fechas de ingreso y salida; el tipo de habitacion; plaza de aparcamiento; plan de comidas, etc. Las razones tipicas de las cancelaciones incluyen varios factores, a veces se facilita con la opcion de hacerlo gratis o a un bajo costo, que es beneficioso para el cliente pero no asi para los hoteles. ¿Se podrá predecir si el cliente cumplirá o cancelará la reserva? 

## Variables

Booking_ID : identificador único de cada reserva

no_of_adults : Número de adultos

no_of_children : Número de niños

no_of_weekend_nights : Número de noches de fin de semana (sábado o domingo) que el huésped se hospedó o reservó para quedarse en el hotel

type_of_meal_plan : Tipo de plan de comidas reservado por el cliente

required_car_parking_space : ¿Necesita el cliente una plaza de aparcamiento? (0 - No, 1- Sí)

room_type_reservad : Tipo de habitación reservada por el cliente. Los valores son cifrados (codificados) por INN Hotels.

lead_time : Número de días entre la fecha de reserva y la fecha de llegada

arribo_año : Año de la fecha de llegada

llegada_mes : Mes de la fecha de llegada

fecha_llegada : Fecha del mes

market_segment_type : Designación del segmento de mercado.

invitado_repetido: ¿El cliente es un invitado repetido ? (0 - No, 1- Sí)

no_of_previous_cancellations : Número de reservas anteriores que fueron canceladas por el cliente antes de la reserva actual

no_of_previous_bookings_not_canceled : Número de reservas anteriores no canceladas por el cliente antes de la reserva actual

avg_price_per_room : Precio medio por día de la reserva; Los precios de las habitaciones son dinámicos. (en euros)

no_of_special_requests : Número total de solicitudes especiales realizadas por el cliente (por ejemplo, piso alto, vista desde la habitación, etc.)

booking_status : Indicador que indica si la reserva fue cancelada o no.

#### Las variables mas importantes segun el punto de vista del analisis podran ser: "room_type_reservad", "no_of_previous_cancellations" para saber si el cliente cancelo reservas anteriormente, "market_segment_type", "no_of_previous_bookings_not_canceled", "booking_status".

Fuente: Kaggle.com

https://www.kaggle.com/datasets/779de8cc626fb0c43e6083e957a3a045463962637600765c74dd400a268430d3?resource=download